# Tarea Almacenamiento y captura de datos

> Integrantes: Thamara López, Fabián Ortega

Realice un proceso de web scraping para el sitio https://jetsmart.com/cl/es/. Para ello realice las siguientes tareas:  

1) Utilizar Selenium para emular un navegador y cargar la página requerida (1 punto).  
2) Buscar vuelos desde Santiago a Arica del 1 de febrero al 28 de febrero utilizando los campos de inputs del buscador de la página (1 punto).
3) Una vez en la página de resultados, extraiga la información de precios y fechas de vuelos de ida y de vuelta para todos los días de febrero y genere un dataframe con la información de precios y fechas (2 puntos).  
4) Crear una base de datos sqlite con la información del dataframe (1 punto).  
5) Realizar una consulta SQL para seleccionar los vuelos de ida y vuelta con el precio total mínimo (1 punto).  

Utilizando la API de Twitter, realice las siguientes tareas:

1) Extraer los últimos 1.000 tweets de tu usuario preferido que escriba en español (1 punto)
2) Hacer un análisis de frecuencia de palabras (100 palabras más frecuentes), habiendo removido primero stopwords y usando tokenización en español (2 puntos).
3) Extraer 100 usuarios con perfil público que son seguidos por su usuario de preferencia y extraer los últimos 10 tweets de cada uno de esos usuarios (1 punto).
4) Hacer un análisis de frecuencia de palabras igual al de la parte b) (100 palabras más frecuentes), de los tweets extraídos en la parte c) (2 puntos).   




Cada pregunta tiene una escala de 1 a 7. El promedio de ambas preguntas resulta en la nota final de la tarea. 

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/fabianalexis/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [10]:
options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver.implicitly_wait(3)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/fabianalexis/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [13]:
btn_close = '/html/body/div[1]/main/div[7]/div/div[6]/div/div/div[1]/button'

In [20]:
driver.get('https://jetsmart.com/cl/es/')
time.sleep(5)
try: 
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH,
                                        btn_close)))\
        .click()
except: 
    pass

In [22]:
origen = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[1]/input[1]'
scl = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[2]/div[2]/ul[1]/li[1]/ul/li[1]'
destino = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[1]/input[2]'
ar = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[2]/div[2]/ul[1]/li[1]/ul/li[2]'

try: 
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH,
                                        origen)))\
        .click()
    time.sleep(5)
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH,
                                        scl)))\
        .click()


except:
    raise('Error')

In [9]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.110)
